https://jira.x5.ru/browse/CVMUPG-1085

In [1]:
name = 'CVMUPG-1085_danone_add'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
import datetime
from datetime import timedelta
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 
from segmentation import *
from spark import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/01 12:45:50 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [5]:
usecase = ['cross', 'upgrade', 'ump']
dt = datetime.date.today()

In [6]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

In [7]:
customers_usecase.count() # 15 862 125

15862125

### Проверяем на доступность отобранных гостей в определенную дату

In [8]:
check_date = '2022-04-01'

In [9]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:12


In [10]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:05


In [11]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [12]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [13]:
seg_sms.write.parquet('temp1085_1', mode='overwrite')
seg_sms = spark.read.parquet('temp1085_1')

In [14]:
seg_sms.count() # 10 273 447

10273447

# seg1

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [15]:
start_date1 = datetime.date(2021, 8, 25)
end_date1 = datetime.date(2021, 11, 24)
start_date2 = datetime.date(2021, 11, 25)
end_date2 = datetime.date(2022, 1, 24)

In [16]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [17]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [18]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [19]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [20]:
plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001']
# syntethic_category_id = [51,26,52]
# plu_brand_code = ['5725', 'M082']
# plu_not_in = [4138521]

In [21]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [22]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [23]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [24]:
# те, кто покупал во 2 период
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [25]:
# те, кто не покупал во 2 период
seg2 = seg_sms.join(seg1, on='customer_rk', how='left_anti')

In [26]:
checks_headers2 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [27]:
loyalty_cards2 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards2.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms, on='customer_rk', how='inner')

In [28]:
checks_headers_tc5_2 = checks_headers2.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [29]:
checks_items2 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [30]:
plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001']
# syntethic_category_id = [51,26,52]
# plu_brand_code = ['5725', 'M082']
# plu_not_in = [4138521]

In [31]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [32]:
checks_items2 = checks_items2.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [33]:
checks_tc5_2 = checks_items2.join(checks_headers_tc5_2, 'rtl_txn_id', how='inner')

In [34]:
# те, кто покупал в 1 период даниссимо
seg3 = (checks_tc5_2.select('customer_rk')
                  .distinct()
      )

In [35]:
# те, кто не покупал во 2 период даниссимо, хотя раньше покупали
seg = seg3.join(seg2, on='customer_rk', how='inner')

In [36]:
seg.write.parquet('temp1085_seg1', mode='overwrite')
seg = spark.read.parquet('temp1085_seg1')

In [37]:
seg.count()

723476

In [38]:
seg_pd1 = seg.toPandas()

In [39]:
len(seg_pd1)

723476

In [40]:
seg_no = 'seg1'

In [41]:
seg_pd1.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [115]:
seg_pd1 = seg_pd1.astype('str')

cross, upgrade, ump = get_usecases(seg_pd1\
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross), len(upgrade), len(ump), 
              len(cross)+len(upgrade)+len(ump)))

cross: 287629, upgrade: 288101, ump: 147746 (all: 723476)


# seg2

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [42]:
start_date1 = datetime.date(2021, 8, 25)
end_date1 = datetime.date(2021, 11, 24)
start_date2 = datetime.date(2021, 11, 25)
end_date2 = datetime.date(2022, 1, 24)

In [43]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [44]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg, on='customer_rk', how='inner')

In [45]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [46]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date2, end_date2))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [47]:
plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001']
# syntethic_category_id = [51,26,52]
plu_brand_code = ['7532']
# plu_not_in = [4138521]

In [48]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [49]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [50]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [51]:
# те, кто покупал во 2 период
seg1 = (checks_tc5.select('customer_rk')
                  .distinct()
      )

In [52]:
# те, кто не покупал во 2 период
seg2 = seg.join(seg1, on='customer_rk', how='left_anti')

In [53]:
checks_headers2 = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [54]:
loyalty_cards2 = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards2.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg, on='customer_rk', how='inner')

In [55]:
checks_headers_tc5_2 = checks_headers2.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [56]:
checks_items2 = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


In [57]:
plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001']
# syntethic_category_id = [51,26,52]
plu_brand_code = ['7532']
# plu_not_in = [4138521]

In [58]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
                 .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
                 .filter(F.col('plu_brand_code').isin(plu_brand_code))
#                  .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [59]:
checks_items2 = checks_items2.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [60]:
checks_tc5_2 = checks_items2.join(checks_headers_tc5_2, 'rtl_txn_id', how='inner')

In [61]:
# те, кто покупал в 1 период даниссимо
seg3 = (checks_tc5_2.select('customer_rk')
                  .distinct()
      )

In [62]:
# те, кто не покупал во 2 период даниссимо, хотя раньше покупали
seg_2 = seg3.join(seg2, on='customer_rk', how='inner')

In [63]:
seg_2.write.parquet('temp1085_seg2', mode='overwrite')
seg_2 = spark.read.parquet('temp1085_seg2')

[Stage 103:(207 + 3) / 210][Stage 104:(133 + 7) / 140][Stage 105:(375 + 52) / 895]]]]]22/02/01 14:50:07 ERROR cluster.YarnScheduler: Lost executor 406 on mn-hd0192.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
[Stage 116:====================================>               (142 + 58) / 200]4726]]]22/02/01 15:01:48 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.56:48798 is closed
[Stage 116:=====================================>              (144 + 56) / 200]22/02/01 15:01:50 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.56:43026 is closed
[Stage 116:==============================================>     (180 + 20) / 200]22/02/01 15:02:38 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.56:51640 is cl

In [64]:
seg_2.count()

191265

In [65]:
seg_pd2 = seg_2.toPandas()

In [66]:
len(seg_pd2)

191265

In [67]:
seg_no = 'seg2'

In [68]:
seg_pd2.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [111]:
seg_pd2 = seg_pd2.astype('str')

cross, upgrade, ump = get_usecases(seg_pd2)

print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross), len(upgrade), len(ump), 
              len(cross)+len(upgrade)+len(ump)))

### Делаем разбивку полученных сегментов по макрорегионам покупок

### seg

In [69]:
start_date1 = datetime.date(2021, 10, 25)
end_date1 = datetime.date(2022, 1, 24)

In [70]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [71]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg, on='customer_rk', how='inner')

In [72]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [78]:
checks_headers_tc5 = checks_headers_tc5.select('customer_rk', 'store_id').distinct()

In [76]:
macroregions = (spark.table(DIM_STORE)
                .filter(F.col('valid_to_dttm')==datetime.datetime(5999, 1, 1, 0, 0))
                .select('store_id', 'macroregion_dk', 'macroregion_nm'))

In [79]:
seg_for_macro = checks_headers_tc5.join(macroregions, 'store_id', 'left')

In [81]:
seg_macro = (seg_for_macro.groupby(['macroregion_dk', 'macroregion_nm'])
             .agg(F.countDistinct('customer_rk').alias('qty'))
            )

In [82]:
seg_macro.write.parquet('temp1085_seg3', mode='overwrite')
seg_macro = spark.read.parquet('temp1085_seg3')

In [92]:
seg_macro_pd = seg_macro.toPandas()

In [100]:
seg_macro_pd = seg_macro_pd.sort_values('qty', ascending=False).reset_index(drop=True)

In [101]:
seg_macro_pd.to_excel('seg1_macro.xlsx', index=False)

### seg_2

In [83]:
start_date1 = datetime.date(2021, 10, 25)
end_date1 = datetime.date(2022, 1, 24)

In [84]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date1, end_date1))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [85]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_2, on='customer_rk', how='inner')

In [86]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [87]:
checks_headers_tc5 = checks_headers_tc5.select('customer_rk', 'store_id').distinct()

In [88]:
macroregions = (spark.table(DIM_STORE)
                .filter(F.col('valid_to_dttm')==datetime.datetime(5999, 1, 1, 0, 0))
                .select('store_id', 'macroregion_dk', 'macroregion_nm'))

In [89]:
seg_for_macro = checks_headers_tc5.join(macroregions, 'store_id', 'left')

In [90]:
seg_macro_2 = (seg_for_macro.groupby(['macroregion_dk', 'macroregion_nm'])
             .agg(F.countDistinct('customer_rk').alias('qty'))
            )

In [91]:
seg_macro_2.write.parquet('temp1085_seg4', mode='overwrite')
seg_macro_2 = spark.read.parquet('temp1085_seg4')

In [103]:
seg_macro_pd2 = seg_macro_2.toPandas()

In [108]:
seg_macro_pd2

,macroregion_dk,macroregion_nm,qty
0,MRDCT,Москва,67567
1,MRDCN,Центр,34334
2,MRDND,Северо-Запад,26596
3,MRDVN,Волга-Север,19591
4,MRDUG,Юг,17052
5,MRDVO,Волга,15425
6,MRDSB,Сибирь,14137
7,MRDUR,Урал,12473
8,MRDUK,Северный Кавказ,11615
9,MRDUU,Южный Урал,7824


In [105]:
seg_macro_pd2 = seg_macro_pd2.sort_values('qty', ascending=False).reset_index(drop=True)

In [107]:
seg_macro_pd2.to_excel('seg2_macro.xlsx', index=False)